In [1]:
import numpy as np
import pandas as pd
import spiceypy as spice
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.constants import c
from astropy.time import Time
from astroquery.jplhorizons import Horizons
import rebound
import random

In [2]:
def Furnisher(k):
    '''
    This function is used to load all kernels needed in an operation.
    Comment out kernels not in use and add the ones in use.
    
    Arguments: NA
    Returns: NA
    
    '''
    spice.kclear()
    spice.furnsh('/Users/user/Downloads/naif0008.tls.txt')
    if k == '310+341+435':
            spice.furnsh('/Users/user/Downloads/jup310.bsp')
            spice.furnsh('/Users/user/Downloads/jup341.bsp')
            spice.furnsh('/Users/user/Downloads/de435.bsp')
    elif k == '310+341':
            spice.furnsh('/Users/user/Downloads/jup310.bsp')
            spice.furnsh('/Users/user/Downloads/jup341.bsp')
    elif k == '310+435':
            spice.furnsh('/Users/user/Downloads/de435.bsp')
            spice.furnsh('/Users/user/Downloads/jup310.bsp')
    elif k == '341+435':
            spice.furnsh('/Users/user/Downloads/jup341.bsp')
            spice.furnsh('/Users/user/Downloads/de435.bsp')
    elif k == '310':
            spice.furnsh('/Users/user/Downloads/jup310.bsp')
    elif k == '341':
            spice.furnsh('/Users/user/Downloads/jup341.bsp')
    elif k == '435':
            spice.furnsh('/Users/user/Downloads/de435.bsp')
    pass

In [3]:
def get_spice_function(name,cor,loc):
    """
    This wrapper function automates the creation of objects through the JPL Horizons database. 
    
    Arguments:
    
    name: str
    
    Stipulates the target object in Horizons. The major bodies in the Solar System have an id based on their position.
    Hence '5' refers to Jupiter and '3' to Earth. A single number designator refers to a barycenter and a designator
    such as '599' to the planetary center. For minor bodies in the Solar System, the id_type in the Horizons
    must be changed to "minorbody"
    
    cor: str
    
    Refers to the type of correction that the object has. Available arguments are 'NONE', 'LT','LT+S'
    
    loc: str
    
    Designates the location of observation. Names that start with "g@#" refer to barycenters where the number designates the 
    body that the observer is based at. Hence "g@0" refers to the Solar System barycenter. Also takes Earth location designators.
    Observatories are named after their code. Hence, Pan-Starrs observatory is referred as "f51"

    Returns:
    
    get_target_xyz function
    """    
    def get_target_xyz(t):
        """
        Returns the vectors of the Horizons body at a certain time t.
        
        Arguments:
        
        t: days
        
        Julian date of observation
        
        Returns:
    
        xyz: numpy array
        
        A position vector of the observed object
    
        uvw: numpy array
        
        An instantaneous velocity vector of the observed object
        
        radec: numpy array
        
        The right ascension and declination of the observed object
        """
        
        state,lighttime = spice.spkezr(name,t,'J2000',cor,loc)
        pos,lighttime = spice.spkpos(name,t,'J2000',cor,loc)
        range,ra,dec = spice.recrad(pos) 
        xyz = np.array([state[0],state[1],state[2]])/149597870.7#6.68459e-9
        uvw = np.array([state[3],state[4],state[5]])/149597870.7*24.*3600.#*6.68459e-9
        radec = np.array([ra,dec])
        return xyz,uvw,radec*180/np.pi
    return get_target_xyz

In [4]:
# From 432 (km^3/s^2)
GMEarth = 398600.435420
GMMoon = 27068703.151194

GMMercuryB = 22031.780000
GMVenusB = 324858.592000
GMEarthB = 403503.235502
GMMarsB = 42828.375214
GMJupiterB = 126712764.133446
GMSaturnB = 37940585.200000
GMUranusB = 5794556.465752
GMNeptuneB = 6836527.100580
GMSun = 132712440041.939400 # Just Sun
GMSunterr = GMSun + GMMercuryB + GMVenusB + GMEarthB + GMMarsB

# From 310

GMJupiter = 1.266865341960128E+08
GMIo = 5.959924010272514E+03
GMEuropa = 3.202739815114734E+03
GMGanymede = 9.887819980080976E+03
GMCallisto = 7.179304867611079E+03
GMAmalthea = 1.487604677404272E-01
GM10310 = 1.327132332639000E+11

GMSaturnB310 = 3.794058500000000E+07
GMUranusB310 = 5.794548600000000E+06
GMNeptuneB310 = 6.836527100580000E+06
GMJupiterB310 = 1.267127641334463E+08

In [5]:
MMercuryB = GMMercuryB/GMSun
MVenusB = GMVenusB/GMSun
MEarthB = GMEarthB/GMSun
MEarth = GMEarth/GMSun
MMoon = GMMoon/GMSun
MMarsB = GMMarsB/GMSun
MJupiter = (GMJupiterB-GMIo-GMEuropa-GMGanymede-GMCallisto)/GMSun
MSaturnB = GMSaturnB/GMSun
MUranusB = GMUranusB/GMSun
MNeptuneB = GMNeptuneB/GMSun
MIo = GMIo/GMSun
MEuropa = GMEuropa/GMSun
MGanymede = GMGanymede/GMSun
MCallisto = GMCallisto/GMSun
MAmalthea = GMAmalthea/GMSun

In [6]:
Furnisher("310+341+435")

In [7]:
def get_astroquery_function(name,cor,loc):
    """
    This wrapper function automates the creation of objects through the JPL Horizons database. 
    
    Arguments:
    
    name: str
    
    Stipulates the target object in Horizons. The major bodies in the Solar System have an id based on their position.
    Hence '5' refers to Jupiter and '3' to Earth. A single number designator refers to a barycenter and a designator
    such as '599' to the planetary center. For minor bodies in the Solar System, the id_type in the Horizons
    must be changed to "minorbody"
    
    cor: str
    
    Refers to the type of correction that the object has. Available arguments are "geometric","astrometric" and 
    "apparent"
    
    loc: str
    
    Designates the location of observation. Names that start with "g@#" refer to barycenters where the number designates the 
    body that the observer is based at. Hence "g@0" refers to the Solar System barycenter. Also takes Earth location designators.
    Observatories are named after their code. Hence, Pan-Starrs observatory is referred as "f51"

    Returns:
    
    get_target_xyz function
    """    
    def get_target_xyz(t):
        """
        Returns the vectors of the Horizons body at a certain time t.
        
        Arguments:
        
        t: days
        
        Julian date of observation
        
        Returns:
    
        xyz: numpy array
        
        A position vector of the observed object
    
        uvw: numpy array
        
        An instantaneous velocity vector of the observed object
        """
        
        obj = Horizons(id = name, location = loc, epochs = t, id_type = 'majorbody')
        obj1 = obj.vectors(aberrations = cor, refplane = 'earth')
        xyz = np.array([float(obj1['x']),float(obj1['y']),float(obj1['z'])])
        uvw = np.array([float(obj1['vx']),float(obj1['vy']),float(obj1['vz'])])
        obj2 = obj.ephemerides(refsystem = 'J2000', extra_precision = True)
        radec = np.array([float(obj2['RA']),float(obj2['DEC']),float(obj1['range'])])
        return xyz,uvw,radec
    return get_target_xyz 

In [8]:
def CordConv(xyz):
    '''
    This function takes in a position vector of a body relative to an observer and returns a radec.
    
    Arguments:
    
    xyz: numpy array
    
    Should be three values, the x,y,z of the position vector
    
    Returns:
    
    radec: numpy array
    
    Two values, the first for right ascension and the second for declination
    
    '''
    DEC = -(np.arccos(xyz[2]/np.linalg.norm(xyz))-np.pi/2)
    RA = (np.arctan2(xyz[1],xyz[0]))
    while (RA > 2*np.pi):
        RA -= 2*np.pi   
    while (RA < 0):
        RA += 2*np.pi   
    return np.array([RA*180/np.pi,DEC*180/np.pi])

In [38]:
xyzs = []
get_earth = get_astroquery_function('0','geometric','f51')
for j in range(0,len(times)):
    thor = Time(times[j], format='jd', scale='utc')
    thor = thor.tt.value
    F51Sun = get_earth(thor)
    xyzs.append(F51Sun)
    print(j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [65]:
def Integrator2(t,bsolar,msolar,iJovian,body1,body2,Ms):
    '''
    Integrates the system. If body1 or body2 refer to the Jupiter Barycenter, while there are bodies in the Jovian system
    teh function will compute the barycenter in the simulation
    
    Arguments:
    
    bsolar
    msolar
    iJovian
    
    body1 and body2: name in str
    
    The spice names of the bodies that we want to look into to check the error of our integrator
    
     '''
    sim = SimStart3(t,bsolar,msolar,iJovian,Ms)
    year = 365.25 # days
    tmax = 1*year
    sim.integrate(tmax)
    return sim

In [70]:
def Integrator(t,ten,bsolar,msolar,iJovian,Ms,arr):
    '''
    Integrates the system. If body1 or body2 refer to the Jupiter Barycenter, while there are bodies in the Jovian system
    teh function will compute the barycenter in the simulation
    
    Arguments:
    
    bsolar
    msolar
    iJovian
    
    body1 and body2: name in str
    
    The spice names of the bodies that we want to look into to check the error of our integrator
    
     '''
    sim = SimStart2(t,bsolar,msolar,iJovian,Ms,arr)
    year = 365.25 # days
    tmax = (ten - t)/86400 
    sim.integrate(tmax)
    return sim

In [9]:
def FinderSpicef51(name,t):
    '''
    
    Finds the radec of objects in the jovian system from the pansstars observatory. Used to test the correspondence
    of spice and Horizons.
    
    Arguments:
    
    name: str
    
    The name of the object in spice documentation. See Summary_Names.txt
    
    t: float
    
    A julian date
    
    '''
    thor = Time(t, format='jd', scale='utc')
    thor = thor.tt.value
    tstr = "jd " + str(t) + " utc"
    ts = spice.str2et(tstr)
    get_earth = get_astroquery_function('0','geometric','f51')
    get_jup = get_spice_function(name,"NONE","0")
    exyz = get_earth(thor)
    jxyz = get_jup(ts)
    fxyz = LT(ts, -exyz[0], get_jup)
    vectors = fxyz[0] + exyz[0]
    return CordConv(vectors)

In [10]:
def FinderHorizonsf51(name,t):
    '''
    
    Finds the radec of objects in the jovian system from the pansstars observatory from the Horizons data base.
    
    Arguments:
    
    name: str
    
    The name of the object in spice documentation. See Summary_Names.txt
    
    t: float
    
    A julian date
    
    '''
    thor = Time(t, format='jd', scale='utc')
    thor = thor.tt.value
    get_pans = get_astroquery_function(name,'astrometric','f51')
    vecs2 = get_pans(thor)
    return CordConv(vecs2[0])

In [64]:
def SimStart3(t,bsolar,msolar,iJovian,Ms):
    '''
    Starts the simulation. Adds all particles
    
    Arguments:
    
    bsolar: list of strs
    
    The names of the bodies added. Typically: '1','2','3','4', 'Jupiter','Io','Ananke'. 'Jupiter' must be added
    last followed by the satellites in the system
    
    msolar: list of floats
    
    The masses of the bodies in bsolar. Must be at the order of bsolar
    
    iJovian: list, int
    
    The indexes (+1) that the satellites in the solar system have in bsolar
    
    '''
    
    sim = rebound.Simulation()
    k = np.sqrt(Ms*10**9)*(1.49597870700*10**11)**(-1.5) *86400.00000
    sim.G = k**2
    sim.add(m=1.)
    for i in range(0,len(bsolar)):
        get_planet = get_spice_function(bsolar[i],'NONE','SUN')
        xyz,uvw,radec = get_planet(t)
        xyz = xyz
        #uvw = (uvw*24*3600)
        sim.add(m=msolar[i],x=xyz[0],y=xyz[1],z=xyz[2],vx=uvw[0],vy=uvw[1],vz=uvw[2])
    ps = sim.particles
    for j in range(1,len(ps)):
        if j not in iJovian:
            ps[j].calculate_orbit(primary=ps[0])
        elif bsolar[j-1] == 'Moon':
            ps[j].calculate_orbit(primary=ps[bsolar.index('Earth')+1])
        else:
            if 'Jupiter' in bsolar:
                ps[j].calculate_orbit(primary=ps[bsolar.index('Jupiter')+1])
            else:
                ps[j].calculate_orbit(primary=ps[bsolar.index('5')+1])
    sim.move_to_com()
    return sim

In [11]:
def SimStart2(t,bsolar,msolar,iJovian,Ms,arr):
    '''
    Starts the simulation. Adds all particles
    
    Arguments:
    
    bsolar: list of strs
    
    The names of the bodies added. Typically: '1','2','3','4', 'Jupiter','Io','Ananke'. 'Jupiter' must be added
    last followed by the satellites in the system
    
    msolar: list of floats
    
    The masses of the bodies in bsolar. Must be at the order of bsolar
    
    iJovian: list, int
    
    The indexes (+1) that the satellites in the solar system have in bsolar
    
    '''
    
    sim = rebound.Simulation()
    k = np.sqrt(Ms*10**9)*(1.49597870700*10**11)**(-1.5) *86400.00000
    sim.G = k**2
    sim.add(m=1.)
    for i in range(0,len(bsolar)):
        get_planet = get_spice_function(bsolar[i],'NONE','SUN')
        xyz,uvw,radec = get_planet(t)
        xyz = xyz
        #uvw = (uvw*24*3600)
        sim.add(m=msolar[i],x=xyz[0],y=xyz[1],z=xyz[2],vx=uvw[0],vy=uvw[1],vz=uvw[2])
    ps = sim.particles
    len(ps)
    for i in range(0,len(arr)):
        sim.add(a=arr[i][0],e=arr[i][1],inc=arr[i][2],Omega=arr[i][3],omega=arr[i][4],f=arr[i][5],primary=ps[bsolar.index('Jupiter')+1])  
    sim.n_active = len(bsolar)+1
    sim.move_to_com()
    return sim

In [36]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
df = pd.read_excel('/Users/user/wfilter.xlsx', sheetname='Sheet1')
print("Column headings:")
print(df.columns)
IDs = np.array(df['ID'])
times = np.array(df['JDUTC'])

Column headings:
Index(['ID', 'smf-name', 'smf-filepath', 'reduction', 'JDUTC', 'RA', 'Dec',
       'filter', 'exp-time', 'UVx', 'UVy', 'UVz', 'obseqx', 'obseqy', 'obseqz',
       'obseclx', 'obseclsy', 'obseclz'],
      dtype='object')


/anaconda3/lib/python3.6/site-packages/pandas/io/excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [37]:
def Seeder(n):
    arr = []
    for i in range(0,n):
        e = random.uniform(0,1)
        a = random.uniform(0.05,(0.177/(1+e)))
        i = random.uniform(0,np.pi)
        Om = random.uniform(0,2*np.pi)
        om = random.uniform(0,2*np.pi)
        k = random.uniform(0,2*np.pi)
        arr.append(np.array([a,e,i,Om,om,k]))
    return arr

In [63]:
def FinderReboundf51(t,years,arr,times,vecs,IDs):
    '''
    
    Finds the radec of objects in the jovian system from the pansstars observatory a number of years before
    the time of observation and integrates it to the time of observation using rebound.
    
    Arguments:
    
    name: str
    
    The name of the object in spice documentation. See Summary_Names.txt
    
    t: float
    
    A julian date
    
    '''
    tstr = "jd " + str(t) + " utc"
    ts = spice.str2et(tstr)
    get_earth = get_astroquery_function('0','geometric','f51')
    sec_year = 86400*365.25
    sim = Integrator(ts - years*sec_year,ts,['1','2','3','4','6','7','8',
        'Jupiter','Io','Europa','Ganymede','Callisto'],[MMercuryB,MVenusB,
        MEarthB,MMarsB,MSaturnB,MUranusB,MNeptuneB,MJupiter,MIo,MEuropa,MGanymede,
        MCallisto],[9,10,11,12,13,14,15],GMSun,arr)
    ps = sim.particles
    radec = []
    ttemp = ts
    for j in range(0,len(times)):
        temp = "jd " + str(times[j]) + " utc"
        ttemp2 = spice.str2et(temp)
        sim.integrate(sim.t + (ttemp2-ttemp)/86400)
        ps = sim.particles
        ttemp = ttemp2
        F51 = vecs[j][0]
        for i in range(13,len(ps)):
            dist = np.sqrt((ps[i].x-ps[3].x)**2 + (ps[i].y-ps[3].y)**2 + (ps[i].z-ps[3].z)**2)
            ltime = float(((dist*u.AU).to(u.m)/c)/u.s)
            sim.integrate(sim.t - ltime/(24*3600))
            ps = sim.particles
            AnSun = np.array([float(ps[i].x),float(ps[i].y),float(ps[i].z)])
            vectors = AnSun + F51Sun[0]
            radec.append(np.array([CordConv(vectors),times[j],IDs[j]]))
            sim.integrate(sim.t + ltime/(24*3600))
            ps = sim.particles
    return radec

In [233]:
def FinderReboundf51_2(name,t,years,arr,times):
    '''
    
    Finds the radec of objects in the jovian system from the pansstars observatory a number of years before
    the time of observation and integrates it to the time of observation using rebound.
    
    Arguments:
    
    name: str
    
    The name of the object in spice documentation. See Summary_Names.txt
    
    t: float
    
    A julian date
    
    '''
    thor = Time(t, format='jd', scale='utc')
    thor = thor.tt.value
    tstr = "jd " + str(t) + " utc"
    ts = spice.str2et(tstr)
    get_earth = get_astroquery_function('0','geometric','f51')
    sec_year = 86400*365.25
    sim = Integrator(ts - years*sec_year,ts,['1','2','3','4','6','7','8',
        'Jupiter','Io','Europa','Ganymede','Callisto'],[MMercuryB,MVenusB,
        MEarthB,MMarsB,MSaturnB,MUranusB,MNeptuneB,MJupiter,MIo,MEuropa,MGanymede,
        MCallisto],[9,10,11,12,13,14,15],GMSun,arr)
    ttemp = ts
    radec = []
    print(sim.t)
    for j in range(0,len(times)):
        temp = "jd " + str(times[j]) + " utc"
        ttemp2 = spice.str2et(temp)
        cqtime.append(sim.t)
        k = sim.t + (ttemp2-ttemp)/86400
        ttemp = ttemp2
        sim.integrate(k)
        thor = Time(times[j], format='jd', scale='utc')
        thor = thor.tt.value
        ps = sim.particles
        dist = np.sqrt((ps[-1].x-ps[3].x)**2 + (ps[-1].y-ps[3].y)**2 + (ps[-1].z-ps[3].z)**2)
        ltime = float(((dist*u.AU).to(u.m)/c)/u.s)
        k = sim.t - ltime/(24*3600)
        sim.integrate(k)
        ps = sim.particles
        AnSun = np.array([float(ps[-1].x),float(ps[-1].y),float(ps[-1].z)])
        F51Sun = get_earth(thor)
        vectors = AnSun + F51Sun[0]
        radec.append(CordConv(vectors))
        k = sim.t + ltime/(24*3600)
        sim.integrate(k)
        print(j)
    return radec

In [218]:
radec2,cqtime2 = FinderReboundf51_2('Ananke',times[0]-1,1,arr,times)

365.25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275

In [231]:
rad2 = []
for i in range(0,len(times)):
    rad2.append(FinderHorizonsf51('5',times[i]))
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [232]:
m = np.array(rad2)-np.array(radec2)
for i in range(0,len(m)):
    print(m[i])

[-0.95724714 -0.14638101]
[-0.95725188 -0.14637028]
[-0.95728336 -0.14629894]
[-0.95728805 -0.14628831]
[-0.96574017 -0.12603713]
[-0.96574465 -0.12602602]
[-0.96576927 -0.12596495]
[-0.96577371 -0.12595395]
[-0.99232939 -0.04279002]
[-0.99234888 -0.04270999]
[-0.99483045 -0.03183438]
[-0.99483429 -0.03181705]
[-0.99485097 -0.03174175]
[-0.98498848  0.19797815]
[-0.98101342  0.20880172]
[-0.98098741  0.20886992]
[-0.97663516  0.21964281]
[-0.97661861  0.21968236]
[-0.97660689  0.21971033]
[-0.97659052  0.2197494 ]
[-0.9666647   0.24124118]
[-0.96664836  0.24127401]
[-0.96663194  0.24130699]
[-0.96661549  0.24134004]
[-0.94841761  0.27348013]
[-0.94840386  0.27350197]
[-0.94836896  0.2735574 ]
[-0.94835532  0.27357905]
[0.97138997 0.02799154]
[0.97150785 0.02781201]
[0.97162514 0.02763319]
[0.97163179 0.02762304]
[0.97174338 0.02745271]
[0.98294295 0.00968604]
[0.98307447 0.00946899]
[0.98320423 0.00925456]
[0.98333342 0.00904078]
[ 1.04108649 -0.30940671]
[ 1.04103556 -0.30961613]
[ 1.

[ 1.29496889 -0.00760764]
[ 1.29489609 -0.00747303]
[ 1.29486716 -0.00741958]
[ 1.29479411 -0.0072847 ]
[ 1.29476487 -0.00723076]
[ 1.29469162 -0.0070957 ]
[ 1.29466213 -0.00704137]
[1.25907549 0.05212577]
[1.25905544 0.0521562 ]
[1.25904892 0.05216608]
[1.25904225 0.0521762 ]
[1.25894236 0.05232763]
[1.25892221 0.05235816]
[1.25891533 0.05236859]
[1.25890876 0.05237854]
[1.25880904 0.05252953]
[1.25878868 0.05256033]
[1.2587818  0.05257075]
[1.25877531 0.05258055]
[1.25866929 0.05274085]
[1.25864853 0.05277223]
[1.25864202 0.05278207]
[1.24925532 0.06665853]
[1.24923465 0.06668856]
[1.24922791 0.06669835]
[1.24912131 0.0668531 ]
[1.24910072 0.06688298]
[1.24909401 0.06689272]
[1.24899102 0.06704206]
[1.24897039 0.06707194]
[1.24896369 0.06708167]
[1.24886081 0.06723066]
[1.24883997 0.06726083]
[1.24883324 0.06727058]
[1.23743307 0.08335941]
[1.23740478 0.08339829]
[1.23730192 0.08353955]
[1.23727371 0.08357827]
[1.23717172 0.08371819]
[1.23714358 0.08375678]
[1.23704163 0.0838965 ]
[1